In [3]:
import numpy as np 
import pandas as pd

In [4]:
#loading the data
workshop_data = pd.read_csv("C:/Users/abhin/Downloads/workshops_dataset.csv")
students_data = pd.read_csv("C:/Users/abhin/Downloads/student_data.csv")

In [5]:
workshop_data.head()

,title,club,tags
0,CONVenient,"ACM, Manipal","Neural Networks, The Convolution Operation, Co..."
1,Fusion 360 Generative Design Workshop,DronAid,"Fusion 360, Sketching basics, 3D modelling, As..."
2,PCB Designing,DronAid,"PCB Design, KiCad, Basic electronic components..."
3,Astravaganza,"The Astronomy Club, Manipal","Types of Telescope, Dobsonian telescope, Starg..."
4,App Development & Backend Workshop,Apple Developers Group,"React Native, TypeScript, Cross-platform app d..."


In [6]:
students_data.head()

,Name,Reg_No,Branch,Interests
0,Student001,REG0001,Mathematics & Computing,"Probability Models, Data Science, Algorithm De..."
1,Student002,REG0002,Mathematics & Computing,"Algorithm Design, Probability Models, Cryptogr..."
2,Student003,REG0003,Information Technology,"Web Development, Database Management, Cloud Co..."
3,Student004,REG0004,Automobile Engineering,"Automotive Design, EV Technologies, Vehicle Dy..."
4,Student005,REG0005,Aeronautical Engineering,"Aerospace Materials, Propulsion Systems, Space..."


In [7]:
#concatinating branch and interests fields
students_data['Branch_Interests'] = students_data['Branch'] + ', ' + students_data['Interests']

In [8]:
students_data.head()

,Name,Reg_No,Branch,Interests,Branch_Interests
0,Student001,REG0001,Mathematics & Computing,"Probability Models, Data Science, Algorithm De...","Mathematics & Computing, Probability Models, D..."
1,Student002,REG0002,Mathematics & Computing,"Algorithm Design, Probability Models, Cryptogr...","Mathematics & Computing, Algorithm Design, Pro..."
2,Student003,REG0003,Information Technology,"Web Development, Database Management, Cloud Co...","Information Technology, Web Development, Datab..."
3,Student004,REG0004,Automobile Engineering,"Automotive Design, EV Technologies, Vehicle Dy...","Automobile Engineering, Automotive Design, EV ..."
4,Student005,REG0005,Aeronautical Engineering,"Aerospace Materials, Propulsion Systems, Space...","Aeronautical Engineering, Aerospace Materials,..."


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#creating a TfidfVectorizer object
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(workshop_data['tags'])

#transforming user data
user_tfidf_matrix = tfidf.transform(students_data['Branch_Interests'])

#calculating cosine similarity
cosine_sim = cosine_similarity(user_tfidf_matrix, tfidf_matrix)

In [10]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

# Apply stemming to workshop tags and student interests to standardize word forms
# Stemming reduces words to their root form (e.g., "programming" -> "program"),
# ensuring uniformity across the tags and interests. This helps improve matching
# between workshops and student preferences by handling variations in phrasing
# (e.g., "computing" vs. "compute").

workshop_data['tags'] = workshop_data['tags'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))
students_data['Branch_Interests'] = students_data['Branch_Interests'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

In [11]:
# For each student, get the top-k workshops (e.g., k=3)
top_k = 3
recommendations = []

for student_idx in range(cosine_sim.shape[0]):
    # Get sorted indices of workshops based on similarity
    similar_workshops = cosine_sim[student_idx].argsort()[::-1][:top_k]
    recommendations.append(workshop_data.iloc[similar_workshops]['title'].values)

In [12]:
#code for popular workshops

In [13]:
students_data['Recommended_Workshops'] = recommendations

In [14]:
students_data.to_csv('student_recommendations.csv', index=False)

In [15]:
cosine_sim

array([[0.        , 0.16629049, 0.12049193, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.14342952, 0.        ,
        0.        , 0.17926871, 0.16309389, 0.        ],
       [0.        , 0.14647346, 0.10613276, 0.        , 0.        ,
        0.12220177, 0.12086081, 0.        , 0.12633686, 0.        ,
        0.        , 0.15790505, 0.1436578 , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.0777684 ,
        0.        , 0.15233067, 0.        , 0.        , 0.        ,
        0.        , 0.29817767, 0.10160745, 0.        ],
       [0.        , 0.34771954, 0.25195306, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.29991641, 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.22576463, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.29164018, 0.2